In [1]:
from __future__ import division
import numpy as np
import os, sys
import matplotlib
#matplotlib.use('Agg')
%matplotlib nbagg
import matplotlib.pyplot as plt
import pandas as pd
from collections import defaultdict


def getCDF(data):
    xdata = np.sort(data)
    ydata = [i/len(xdata) for i in range(len(xdata))]
    return xdata, ydata

## LOAD 1 HOUR/LATE NIGHT DATA
- 24 JAN 2017
- 2AM-3AM

In [3]:
df = pd.read_csv("../data/dns_data_cs/1hour-night/split_hour_00002_20170124015900.csv", sep="|", error_bad_lines=False,
                names=['time', 'srcip', 'dstip', 'dnsquery']).fillna('')

In [8]:
df.head()

,time,srcip,dstip,dnsquery
0,0.004084000,10.9.186.52,10.8.0.5,www.youtube.com
1,0.004886000,10.9.186.52,10.8.0.6,www.youtube.com
2,0.007861000,10.9.231.134,10.8.0.5,instagram.com
3,0.010363000,10.9.186.52,10.8.0.5,youtube-ui.l.google.com
4,0.026029000,10.9.185.235,10.8.0.5,guzzoni.apple.com


In [48]:
df [ df['dnsquery'].str.contains('csgsmtp202l') ]

,time,srcip,dstip,dnsquery
285,0.797733000,140.180.223.155,128.112.129.7,csgsmtp202l.Princeton.EDU
286,0.797775000,140.180.223.155,128.112.129.7,csgsmtp202l.Princeton.EDU
287,0.799954000,140.180.223.155,128.112.129.209,csgsmtp202l.Princeton.EDU.Princeton.EDU
288,0.800001000,140.180.223.155,128.112.129.209,csgsmtp202l.Princeton.EDU.Princeton.EDU
18431,59.020377000,10.6.52.11,128.112.129.7,csgsmtp202l.Princeton.edu
19502,60.923012000,140.180.223.155,128.112.129.209,csgsmtp202l.Princeton.EDU
19503,60.923023000,140.180.223.155,128.112.129.209,csgsmtp202l.Princeton.EDU
19504,60.924562000,140.180.223.155,128.112.129.7,csgsmtp202l.Princeton.EDU.Princeton.EDU
19505,60.924607000,140.180.223.155,128.112.129.7,csgsmtp202l.Princeton.EDU.Princeton.EDU
43276,120.968245000,140.180.223.155,128.112.129.7,csgsmtp202l.Princeton.EDU


In [10]:
len(df)

1099486

In [9]:
len(df['srcip'].unique())

8138

In [47]:
def reduce_domain(domain, level):
    tmp = domain.split('.')
    out = '.'.join( tmp[-level:] )
    return out
    
reduce_domain('www.youtube.com', 1)

'com'